<a href="https://colab.research.google.com/github/rickyjasso/Multiagentes/blob/main/M1%20Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 5.1 MB/s 
     |████████████████████████████████| 60 kB 5.6 MB/s 


In [83]:
# Importar de mesa las clases Agent y Model
from mesa import Agent, Model 

# MultiGrid para tener multiples agentes en la misma celda
from mesa.space import MultiGrid

# Los agentes se activan de manera simultanea
from mesa.time import SimultaneousActivation

# DataCollector nos ayuda a tener todo el grid completo.
from mesa.datacollection import DataCollector 

# Matplotlib nos ayuda a graficar
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# numpy y pandas se utilizan para numeros
import numpy as np
import pandas as pd

# time y datetime los utilizamos para ver el tiempo de ejecución y compararlo con nuestro parametro de tiempo maximo
import time
import datetime

In [84]:
def get_grid(model):

    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
          if agent.unique_id[2] == 0:
            grid[x][y] = agent.cleaner
          else:
            grid[x][y] = agent.clean

    return grid



class CleanerAgent(Agent):

    def random_move(self):

        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, True, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

    def __init__(self, unique_id, model):

        super().__init__(unique_id, model)  
        self.cleaner = 2    
    
    def step(self):

        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True,
            )
        
        for neighbor in neighbours:
          if neighbor.unique_id[0] == self.pos[0] and neighbor.unique_id[1] == self.pos[1] and neighbor.unique_id[2] == 1:
            neighbor.clean = 0        
    
    def advance(self):
        self.random_move()

class FloorAgent(Agent):
    def __init__(self, unique_id, model, isDirty):
        super().__init__(unique_id, model)
        #self.clean = np.random.choice([0,1])
        self.clean = isDirty
    
class CleanerModel(Model):
    def __init__(self, width, height, percentage, numAgents):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)

        #Calcular el numero total de celdas que estaran sucias
        grid = width * height
        numPercent = percentage/100
        total = int(grid * numPercent)
        print(total)
        
        # Asignar aleatoreamente las celdas sucias
        dirtyList = np.zeros((width, height),int)
        x = 0
        while x < total:
          posx = np.random.randint(0,width-1)
          posy = np.random.randint(0,height-1)
          if dirtyList[posx][posy] == 0:
            dirtyList[posx][posy] = 1
            x+=1

        print(dirtyList)

        for (content, x, y) in self.grid.coord_iter():
          # Se crea un agente de tipo limpiador N veces, N siendo el numero que se le asigno a NUM_AGENTS
          if numAgents > 0:
            a = CleanerAgent((x, y, 0), self)
            self.grid.place_agent(a, (1, 1))
            self.schedule.add(a)
            numAgents = numAgents - 1

          # Si el valor de dirtyList es 1, la variable .clean de la clase FloorAgent sera 1, lo cual saldra marcado en el grid
          b = FloorAgent((x, y, 1), self, dirtyList[x][y])
          self.grid.place_agent(b, (x, y))
          self.schedule.add(b)

        
        # Aquí definimos el DataCollector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    def noDirty(self):
      retval = 0
      cleanFloor = 0
      # Va de celda en celda checando si el agente es un piso, y si lo es, si el piso esta sucio o limpio,
      # se suma 1 a cleanFloor y cuando cleanFloor sea el mismo numero que el numero de celdas, el piso esta totalmente limpio
      for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
          if agent.unique_id[2] == 1 and agent.clean == 0:
            cleanFloor += 1

      if (self.grid.width * self.grid.height) == cleanFloor:
        retval = 1
      else:
        retval = (cleanFloor / (self.grid.width * self.grid.height))

      return retval




In [119]:
# Definimos el tamaño del Grid
GRID_SIZE = 20

# Definimos el porcentaje de celdas que estaran sucias
PERCENTAGE = 40
 
# Definimos el numero de agentes que estaran trabajando para limpiar
NUM_AGENTS = 30

# Definimos el tiempo maximo de ejecución
MAX_RUN = 0.1

# Registramos el tiempo de inicio
start_time = time.time()

# Definimos la variable para movimientos de los agentes
movimientos = 0

model = CleanerModel(GRID_SIZE, GRID_SIZE, PERCENTAGE, NUM_AGENTS)


while(time.time()-start_time) < MAX_RUN and model.noDirty() != 1:
  movimientos += 1
  model.step()


# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

print('Porcentaje de celdas limpias después del termino de la simulación:', model.noDirty() * 100, '%')

print('Movimientos totales para completar limpieza: ', movimientos)

160
[[1 1 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 1 0]
 [1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 0]
 [0 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0]
 [1 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 1 1 1 0]
 [0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0]
 [0 1 0 1 0 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0]
 [0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0]
 [1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0]
 [1 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0]
 [0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0]
 [1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 1 0]
 [1 1 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0]
 [1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 0]
 [0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Tiempo de ejecución: 0:00:00.100928
Porcentaje de celdas limpias después del termino de la simulación: 91.5 %
Movimientos totales para 

In [121]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [122]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Greens)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=movimientos)

In [123]:
anim

**INFORME DE LO OBSERVADO**


---


La cantidad de agentes afecta, logicamente, el tiempo que tarda el programa en correr, y en terminar. Entre menos agentes tengas, mas movimientos se tendran que hacer para limpiar todas las celdas del grid. Que los agentes realizen mas movimientos significa que el programa tardara mas en correr ya que el programa tiene que hacer mas generaciones para todos los movimientos. Entre mas agentes tengas, mas facil sera de limpiar todo y menos movimientos se realizaran.

Con los numeros dados, de GRID_SIZE: 20, PERCENTAGE: 40 y NUM_AGENTS: 30, se puede apreciar como si le ponemos un tiempo maximo de 0.1, el programa no logra limpiar el 100% de las celdas, pero si le subimos el tiempo maximo a 0.3, si lo logra.